In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import torch
import torchvision
import tqdm
import pandas as pd
import PIL.Image as Image
print(torch.__version__)
!python --version

In [0]:
!pip3 install kaggle

In [0]:
!mkdir .kaggle

api_token = {"username":"aniruddhk123","key":"GET YOUR OWN KEY"}
import json
import zipfile
import os
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
    
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v data
!kaggle competitions download -c nyu-cv-fall-2018
os.chdir('data/competitions/nyu-cv-fall-2018')
for file in os.listdir():
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall()
    zip_ref.close()

In [0]:
####---------------data.py##################
from __future__ import print_function
import zipfile
import os

import torchvision.transforms as transforms

# once the images are loaded, how do we pre-process them before being passed into the network
# by default, we resize the images to 32 x 32 in size
# and normalize them to mean = 0 and standard-deviation = 1 based on statistics collected from
# the training set
train_transforms = transforms.Compose([
    transforms.RandomAffine(degrees=40,translate=(.3,.3),scale=(.8,1.2)),
    transforms.ColorJitter(brightness=.8),
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))])

val_transforms = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))])


def initialize_data(folder):
    train_zip = folder + '/train_images.zip'
    test_zip = folder + '/test_images.zip'
    if not os.path.exists(train_zip) or not os.path.exists(test_zip):
        raise(RuntimeError("Could not find " + train_zip + " and " + test_zip
              + ', please download them from https://www.kaggle.com/c/nyu-cv-fall-2017/data '))
    # extract train_data.zip to train_data
    train_folder = folder + '/train_images'
    if not os.path.isdir(train_folder):
        print(train_folder + ' not found, extracting ' + train_zip)
        zip_ref = zipfile.ZipFile(train_zip, 'r')
        zip_ref.extractall(folder)
        zip_ref.close()
    # extract test_data.zip to test_data
    test_folder = folder + '/test_images'
    if not os.path.isdir(test_folder):
        print(test_folder + ' not found, extracting ' + test_zip)
        zip_ref = zipfile.ZipFile(test_zip, 'r')
        zip_ref.extractall(folder)
        zip_ref.close()

    # make validation_data by using images 00000*, 00001* and 00002* in each class
    val_folder = folder + '/val_images'
    if not os.path.isdir(val_folder):
        print(val_folder + ' not found, making a validation set')
        os.mkdir(val_folder)
        for dirs in os.listdir(train_folder):
            if dirs.startswith('000'):
                os.mkdir(val_folder + '/' + dirs)
                for f in os.listdir(train_folder + '/' + dirs):
                    if f.startswith('00000') or f.startswith('00001') or f.startswith('00002'):
                        # move file to validation folder
                        os.rename(train_folder + '/' + dirs + '/' + f, val_folder + '/' + dirs + '/' + f)


In [0]:
########--------------main.py PART-1----------------###########
!kaggle config set -n path -v data
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable


torch.manual_seed(1)
torch.cuda.manual_seed(1)

### Data Initialization and Loading
data_path = '.'
initialize_data(data_path) # extracts the zip files, makes a validation set


In [0]:
### ------------------Data Augmentation-----------##

def load_pickled_data(file, columns):
        with open(file, mode='rb') as f:
            dataset = pickle.load(f)
        return tuple(map(lambda c: dataset[c], columns))


def pickle_data(x, y, save_loc):
    print("Saving pickle at " + save_loc)
    save = {"features": x, "labels": y}

    with open(save_loc, "wb") as f:
        pickle.dump(save, f)


def pickle_data_from_folder(data_folder, save_loc):
    if not os.path.isdir(data_folder):
        print("Data folder must be a folder and should contains sub folders for each label")
        return

    resize_transform = transforms.Resize((32, 32))
    sub_folders = os.listdir(data_folder)

    count = 0
    for sub_folder in sub_folders:
        sub_folder = os.path.join(data_folder, sub_folder)

        if not os.path.isdir(sub_folder):
            continue
        label = int(sub_folder.split("/")[-1])

        for image in os.listdir(sub_folder):
            count += 1

    save = {"features": np.empty([count, IMG_SIZE, IMG_SIZE, 3], dtype=np.uint8),
            "labels": np.empty([count], dtype=int)}
    i = 0
    for sub_folder in sub_folders:
        sub_folder = os.path.join(data_folder, sub_folder)

        if not os.path.isdir(sub_folder):
            continue
        label = int(sub_folder.split("/")[-1])
        for image in os.listdir(sub_folder):
            image = os.path.join(sub_folder, image)
            pic = Image.open(image)
            pic = resize_transform(pic)
            pic = np.array(pic)
            save["features"][i] = pic
            save["labels"][i] = label
            i += 1

    with open(save_loc, "wb") as f:
        pickle.dump(save, f)





train_pickle_path = "aug_data/train.p"
pickle_data_from_folder("data"+"train_images", train_pickle_path)
print("Saved train.p from original data folder")
print("Now extending data")
x, y = load_pickled_data(train_pickle_path, ["features", "labels"])
extender = Extender(x, y, 1, .75)
x_extended, y_extended = extender.flip()
print("Data extension complete")


In [0]:
import os
import numpy as np
from skimage.transform import rotate, warp, ProjectiveTransform
from torchvision import datasets, transforms
import random
import pickle


class Extender:
    def __init__(self, data_images, data_labels, ratio=0.5, intensity=0.75):
        self.x = data_images
        self.y = data_labels
        self.intensity = intensity
        self.ratio = ratio
        
        # horizontally flipped
        self.horizontally_flippable_classes = [11, 12, 13, 15, 17, 18, 
                                               22, 26, 30, 35]
        
        # vertically flipped
        self.vertically_flippable_classes = np.array([1, 5, 12, 15, 17])
        
        # flipped both vertically and horizontally
        self.both_flippable = [32, 40]
        
        # Tcontains pair which first can be generated by 
        # horizontally flipping the other.
        self.flip_exchangeable = np.array([
            (19, 20),
            (20, 19),
            (33, 34),
            (34, 33),
            (36, 37),
            (37, 36),
            (38, 39),
            (39, 38)
        ])
    
    def extend_and_balance(self, custom_counts=None):
        print("Extending and balancing dataset with intesity", self.intensity)
        x, y = self.flip()
        _, class_counts = np.unique(y, return_counts=True)
        max_count = max(class_counts)
        
        if custom_counts is None:
            total = max_count * NUM_CLASSES
        else:
            total = np.sum(custom_counts)
        
        x_balanced = np.empty([0, x.shape[1], x.shape[2],x.shape[3]],
                              dtype=np.float32)
        y_balanced = np.empty([0], dtype=y.dtype)
        
        for c, class_count in zip(range(NUM_CLASSES), tqdm(class_counts)):
            x_org = (x[y == c] / 255.).astype(np.float32)
            y_org = y[y == c]
            
            x_balanced = np.append(x_balanced, x_org, axis=0)
            
            max_count = max_count if custom_counts is None else custom_counts[c]
            for i in range(max_count // class_count):
                x_mod = self.rotate(x_org)
                x_mod = self.projection_transform(x_mod)
                x_balanced = np.append(x_balanced, x_mod, axis=0)
            
            if max_count % class_count > 0:
                x_mod = self.rotate(x_org[:(max_count % class_count)])
                x_mod = self.projection_transform(x_mod)
            
                x_balanced = np.append(x_balanced, x_mod, axis=0)
            
            extension = np.full(x_balanced.shape[0] - y_balanced.shape[0],
                                c, dtype=y_balanced.dtype)
            y_balanced = np.append(y_balanced, extension)
            
            del x_org
            del y_org
        
        return (x_balanced * 255).astype(np.uint8), y_balanced
    
    def flip(self):
        x = np.empty([0, self.x.shape[1], self.x.shape[2], 
                      self.x.shape[3]], 
                      dtype=self.x.dtype)
        y = np.empty([0], dtype=self.y.dtype)
        
        for c in range(NUM_CLASSES):
            # Add existing data
            x = np.append(x, self.x[self.y == c], axis=0)
            
            if c in self.horizontally_flippable_classes:
                # Flip columns and append
                x = np.append(x, self.x[self.y == c][:, :, ::-1, :], 
                              axis=0)
                
            if c in self.vertically_flippable_classes:
                # Flip rows and append
                x = np.append(x, self.x[self.y == c][:, ::-1, :, :],
                              axis=0)
            
            if c in self.flip_exchangeable[:, 0]:
                flip_c = self.flip_exchangeable[self.flip_exchangeable[:, 0] == c]
                flip_c = flip_c[0][1]
                
                # Flip other class horizontally 
                x = np.append(x, self.x[self.y == flip_c][:, :, ::-1, :], 
                              axis=0)
            
            if c in self.both_flippable:
                # Flip both rows and columns
                x = np.append(x, self.x[self.y == c][:, ::-1, ::-1, :],
                             axis=0)
            
            # Extend y now
            y = np.append(y, np.full(x.shape[0] - y.shape[0], c, 
                                     dtype=int))
        
        return (x, y)
    
    def rotate(self, x):
        indices = np.random.choice(x.shape[0], int(x.shape[0] * self.ratio),
                                   replace=False)
        
        # If we rotate more than 30 degrees, context is lost.
        change = 30. * self.intensity
        x_return = np.empty(x.shape, dtype=x.dtype)
        for i in indices:
            x_return[i] = rotate(x[i], random.uniform(-change, change), mode="edge")
        
        return x_return
    
    def projection_transform(self, x):
        image_size = x.shape[1]
        
        change = image_size * 0.3 * self.intensity
        
        x_return = np.empty(x.shape, dtype=x.dtype)
        
        indices = np.random.choice(x.shape[0], int(x.shape[0] * self.ratio),
                                   replace=False)
        for i in indices:
            changes = []
            for _ in range(8):
                changes.append(random.uniform(-change, change))
            
            transform = ProjectiveTransform()
            transform.estimate(np.array(
                (
                    (changes[0], changes[1]), # top left
                    (changes[2], image_size - changes[3]), # bottom left
                    (image_size - changes[4], changes[5]), # top right
                    (image_size - changes[6], image_size - changes[7]) # bottom right
                )), np.array(
                (
                    (0, 0),
                    (0, image_size),
                    (image_size, 0),
                    (image_size, image_size)
                ))
            )
            
            x_return[i] = warp(x[i], transform, output_shape=(image_size, image_size),
                        order=1, mode="edge")
        
        return x_return

In [0]:
train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(data_path + '/train_images',
                         transform=train_transforms),
    batch_size=256, shuffle=True, num_workers=1)
val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(data_path + '/val_images',
                         transform=val_transforms),
    batch_size=256, shuffle=False, num_workers=1)

In [0]:
############------------model.py--------------############
import torch
import torch.nn as nn
import torch.nn.functional as F

nclasses = 43 # GTSRB as 43 classes


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 3, kernel_size=1)
        self.conv2 = nn.Conv2d(3, 32, kernel_size=3,  padding=1)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        
        self.conv6 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        
      
        self.fc1 = nn.Linear(229376, 1024)
        self.fc2 = nn.Linear(1024, nclasses)

        
    def flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
      
      
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = F.relu(F.max_pool2d(x,3,stride=1,padding=1))
        x = F.dropout(x, training=self.training)
        layer1 = x
        
        x = self.conv4(x)
        x = self.conv5(x)
        x = F.relu(F.max_pool2d(x,3,stride=1,padding=1)) 
        x = F.dropout(x, training=self.training)
        layer2 = x
        
        x = self.conv6(x)
        x = self.conv7(x)
        x = F.relu(F.max_pool2d(x,3,stride=1,padding=1)) 
        x = F.dropout(x, training=self.training)
        layer3 = x
        
        flat_layer1 = layer1.view(-1, self.flat_features(layer1))
        flat_layer2 = layer2.view(-1, self.flat_features(layer2))
        flat_layer3 = layer3.view(-1, self.flat_features(layer3))
#         print( self.flat_features(layer1)+ self.flat_features(layer2)+ self.flat_features(layer3))
        x = torch.cat([flat_layer1, flat_layer2, flat_layer3],1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.dropout(x, training=self.training)
        return F.log_softmax(x)

In [0]:
model=Net()
model

In [0]:
from torchvision import models

model = Net()

state_dict = torch.load('data/competitions/nyu-cv-fall-2018/resnet_100.pth')
model.load_state_dict(state_dict)  
model.cuda()
optimizer = optim.SGD(model.parameters(), lr=.001, momentum=.7)
  
DEVICE = 'cuda' 
def train(epoch):
    model.train()
    correct=0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    with open('train_acc.csv', 'a') as f:
        f.write("{},{}\n".format(round(100.*correct.numpy()/len(train_loader.dataset),2),epoch))
        
    
    
def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
      with torch.no_grad():
        data, target = data.to(DEVICE), target.to(DEVICE)
        data, target = Variable(data), Variable(target)
        output = model(data)
        validation_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    with open('val_acc.csv', 'a') as f:
        f.write("{},{}\n".format(round(100.*correct.numpy()/len(val_loader.dataset),2),epoch))


In [0]:
for epoch in range(101, 201):
    train(epoch)
    validation()
    model_file = 'resnet_' + str(epoch) + '.pth'
    torch.save(model.state_dict(), model_file)
    print('\nSaved model to ' + model_file + '. You can run `python evaluate.py ' + model_file + '` to generate the Kaggle formatted csv file')


In [0]:
wetorch.cuda.is_available()

In [0]:
from skimage.transform import rotate, warp, ProjectiveTransform
import random

class Extender:
    def __init__(self, data_images, data_labels, ratio=0.5, intensity=0.75):
        self.x = data_images
        self.y = data_labels
        self.intensity = intensity
        self.ratio = ratio
        
        # horizontally flipped for new images
       
        self.horizontally_flippable_classes = [11, 12, 13, 15, 17, 18, 
                                               22, 26, 30, 35]
        
        # vertically flipped for new images
        self.vertically_flippable_classes = np.array([1, 5, 12, 15, 17])
        
        # flipped both vertically and horizontally to get same image
        self.both_flippable = [32, 40]
        
        # These ones contains pair which first can be generated by 
        self.flip_exchangeable = np.array([
            (19, 20),
            (20, 19),
            (33, 34),
            (34, 33),
            (36, 37),
            (37, 36),
            (38, 39),
            (39, 38)
        ])
    
    def extend_and_balance(self, custom_counts=None):
        print("Extending and balancing dataset with intesity", self.intensity)
        x, y = self.flip()
        _, class_counts = np.unique(y, return_counts=True)
        max_count = max(class_counts)
        
        if custom_counts is None:
            total = max_count * 43
        else:
            total = np.sum(custom_counts)
        
        x_balanced = np.empty([0, x.shape[1], x.shape[2], x.shape[3]],
                              dtype=np.float32)
        y_balanced = np.empty([0], dtype=y.dtype)
        
        for c, class_count in zip(range(NUM_CLASSES), tqdm(class_counts)):
            x_org = (x[y == c] / 255.).astype(np.float32)
            y_org = y[y == c]
            
            x_balanced = np.append(x_balanced, x_org, axis=0)
            
            max_count = max_count if custom_counts is None else custom_counts[c]
            for i in range(max_count // class_count):
                x_mod = self.rotate(x_org)
                x_mod = self.projection_transform(x_mod)
                x_balanced = np.append(x_balanced, x_mod, axis=0)
            
            if max_count % class_count > 0:
                x_mod = self.rotate(x_org[:(max_count % class_count)])
                x_mod = self.projection_transform(x_mod)
            
                x_balanced = np.append(x_balanced, x_mod, axis=0)
            
            extension = np.full(x_balanced.shape[0] - y_balanced.shape[0],
                                c, dtype=y_balanced.dtype)
            y_balanced = np.append(y_balanced, extension)
            
            del x_org
            del y_org
        
        return (x_balanced * 255).astype(np.uint8), y_balanced
    
    def flip(self):
        x = np.empty([0, self.x.shape[1], self.x.shape[2], 
                      self.x.shape[3]], 
                      dtype=self.x.dtype)
        y = np.empty([0], dtype=self.y.dtype)
        
        for c in range(43):
            # Add existing data
            x = np.append(x, self.x[self.y == c], axis=0)
            
            if c in self.horizontally_flippable_classes:
                # Flip columns and append
                x = np.append(x, self.x[self.y == c][:, :, ::-1, :], 
                              axis=0)
                
            if c in self.vertically_flippable_classes:
                # Flip rows and append
                x = np.append(x, self.x[self.y == c][:, ::-1, :, :],
                              axis=0)
            
            if c in self.flip_exchangeable[:, 0]:
                flip_c = self.flip_exchangeable[self.flip_exchangeable[:, 0] == c]
                flip_c = flip_c[0][1]
                
                # Flip other class horizontally 
                x = np.append(x, self.x[self.y == flip_c][:, :, ::-1, :], 
                              axis=0)
            
            if c in self.both_flippable:
                # Flip both rows and columns
                x = np.append(x, self.x[self.y == c][:, ::-1, ::-1, :],
                             axis=0)
            
            # Extend y now
            y = np.append(y, np.full(x.shape[0] - y.shape[0], c, 
                                     dtype=int))
        
        return (x, y)
    
    def rotate(self, x):
        indices = np.random.choice(x.shape[0], int(x.shape[0] * self.ratio),
                                   replace=False)
        
        # If we rotate more than 30 degrees, context is lost.
        change = 30. * self.intensity
        x_return = np.empty(x.shape, dtype=x.dtype)
        for i in indices:
            x_return[i] = rotate(x[i], random.uniform(-change, change), mode="edge")
        
        return x_return
    
    def projection_transform(self, x):
        image_size = x.shape[1]
        
        change = image_size * 0.3 * self.intensity
        
        x_return = np.empty(x.shape, dtype=x.dtype)
        
        indices = np.random.choice(x.shape[0], int(x.shape[0] * self.ratio),
                                   replace=False)
        for i in indices:
            changes = []
            for _ in range(8):
                changes.append(random.uniform(-change, change))
            
            transform = ProjectiveTransform()
            transform.estimate(np.array(
                (
                    (changes[0], changes[1]), # top left
                    (changes[2], image_size - changes[3]), # bottom left
                    (image_size - changes[4], changes[5]), # top right
                    (image_size - changes[6], image_size - changes[7]) # bottom right
                )), np.array(
                (
                    (0, 0),
                    (0, image_size),
                    (image_size, 0),
                    (image_size, image_size)
                ))
            )
            
            x_return[i] = warp(x[i], transform, output_shape=(image_size, image_size),
                        order=1, mode="edge")
        
        return x_return

In [0]:
torch.cuda.empty_cache()